In [1]:
import h5py
import pickle
import torch as pt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from SymNet import SymNet

In [2]:
# Load the necessary data
# Load the data
dxNN = np.load("nnJumpLatVecs.npy")
RtoSiteInd = np.load("RtoSiteInd.npy")
SiteIndtoR = np.load("SiteIndtoR.npy")
GpermNNIdx = np.load("GroupNNpermutations.npy")

NNsiteList = np.load("NNsites_sitewise.npy")
N_ngb = NNsiteList.shape[0]

with open("GroupOpsIndices.pkl", "rb") as fl:
    GIndtoGDict = pickle.load(fl)

with open("supercellBCC.pkl", "rb") as fl:
    superBCC = pickle.load(fl)

crys = superBCC.crys
N_units = 8

with open("jnetBCC.pkl", "rb") as fl:
    jnetBCC = pickle.load(fl)

In [3]:
pt.cuda.device_count()

/home/soham/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


0

In [4]:
if pt.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = pt.device("cpu")
print(device)

cpu


In [7]:
GnnPerms = pt.tensor(GpermNNIdx).long().to(device)
NNsites = pt.tensor(NNsiteList).long().to(device)

Nlayers = 3
NchOuts = [2, 2, 1]

# Make the diagonal G matrix for transforming vector kernels
Ng = GnnPerms.shape[0]
Ndim = 3
gdiags = pt.zeros(Ng*Ndim, Ng*Ndim).double()
for gInd, g in GIndtoGDict.items():
    rowStart = gInd * Ndim
    rowEnd = (gInd + 1) * Ndim
    gdiags[rowStart : rowEnd, rowStart : rowEnd] = pt.tensor(g.cartrot).double()

gdiags = gdiags.to(device)

In [6]:
NNsiteList.shape

(9, 512)

In [8]:
TestNet = SymNet(Nlayers, NchOuts, GnnPerms, GIndtoGDict, gdiags, NNsites, Ndim).double().to(device)